# Guided Capstone 2 - NBA Win Model Prediction Using Machine Learning

# Section 0: Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
from datetime import datetime

# Section 1: Data Import and combination

In our investigation, our attempt to recreate the prior investigators model will require data of the same time period and features. In the prior investigation, the data used produce their NBA win prediction model covered a 12 season span, between the seasons of 2008 - 2009 to 2019 - 2020. 

Kaggale offers several databases that include the data portion mentioned above, and can be wrangled to reflect the data of the prior investigation 

# Section 1 Data Wrangling

# 1.1 Data Stitching

From Kaggle, separate sets have been acquired for both individual players statistics per game, and team statistics per game. These sets are separated by season.

In [2]:
#Below code appends datasets together based on file names.
start_year = 2008
end_year = 2009

team_stats = pd.DataFrame()
player_stats = pd.DataFrame()

for i in range(12) :
    player_stats_file_path = '/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/player_stats/' + str(start_year) + "-" + str(end_year) + '.csv'
    team_stats_file_path = '/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/team_stats/' + str(start_year) + "-" + str(end_year) + '.csv'
    player_stats = player_stats.append(pd.read_csv(player_stats_file_path))
    team_stats = team_stats.append(pd.read_csv(team_stats_file_path))
    
    start_year = start_year + 1
    end_year = end_year + 1

In [3]:
player_stats.shape

(373069, 26)

In [4]:
player_stats.head()

,Player,Date,Team,Against,Home,MP,FG,FGA,FG%,3P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,GameLink
0,LeBron James,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,36.00,9.0,21.0,0.429,0.0,...,6.0,7.0,6.0,2.0,1.0,3.0,4.0,22.0,-3.0,https://www.basketball-reference.com/boxscores...
1,Delonte West,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,35.29,2.0,4.0,0.500,0.0,...,2.0,2.0,2.0,0.0,0.0,1.0,0.0,6.0,-10.0,https://www.basketball-reference.com/boxscores...
2,Mo Williams,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,31.18,4.0,10.0,0.400,3.0,...,3.0,3.0,2.0,1.0,0.0,4.0,5.0,12.0,3.0,https://www.basketball-reference.com/boxscores...
3,Zydrunas Ilgauskas,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,30.24,3.0,5.0,0.600,0.0,...,5.0,8.0,1.0,0.0,0.0,4.0,3.0,15.0,7.0,https://www.basketball-reference.com/boxscores...
4,Ben Wallace,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,19.03,1.0,1.0,1.000,0.0,...,4.0,4.0,0.0,0.0,1.0,1.0,2.0,2.0,-1.0,https://www.basketball-reference.com/boxscores...


In [5]:
player_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373069 entries, 0 to 27739
Data columns (total 26 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Player    373069 non-null  object 
 1   Date      373069 non-null  object 
 2   Team      373069 non-null  object 
 3   Against   373069 non-null  object 
 4   Home      373069 non-null  int64  
 5   MP        311326 non-null  float64
 6   FG        311326 non-null  float64
 7   FGA       311326 non-null  float64
 8   FG%       295932 non-null  float64
 9   3P        311326 non-null  float64
 10  3PA       311326 non-null  float64
 11  3P%       199196 non-null  float64
 12  FT        311326 non-null  float64
 13  FTA       311326 non-null  float64
 14  FT%       179762 non-null  float64
 15  ORB       311326 non-null  float64
 16  DRB       311326 non-null  float64
 17  TRB       311326 non-null  float64
 18  AST       311326 non-null  float64
 19  STL       311326 non-null  float64
 20  BLK  

In [6]:
team_stats.shape

(15348, 124)

In [7]:
team_stats.head()

,gmDate,seasonType,season,teamWins,teamLosses,teamAbbr,teamLoc,teamRslt,teamDayOff,teamPTS,...,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,opptPoss,opptPace,matchWinner
0,2008-10-28,Regular,2008-09,0,1,CLE,Away,Loss,0.0,85.0,...,96.9,91.5,5.4,0.41,17.57,1.00,0.62,92.9,92.9,BOS
1,2008-10-28,Regular,2008-09,0,1,MIL,Away,Loss,0.0,95.0,...,120.3,105.8,14.5,0.48,28.53,1.50,0.50,89.8,89.8,CHI
2,2008-10-28,Regular,2008-09,0,1,POR,Away,Loss,0.0,76.0,...,107.7,85.3,22.4,0.44,18.08,1.27,0.53,89.1,89.1,LAL
3,2008-10-29,Regular,2008-09,1,0,TOR,Away,Win,0.0,95.0,...,99.8,112.8,-13.0,0.37,14.47,0.88,0.18,84.2,84.2,TOR
4,2008-10-29,Regular,2008-09,1,0,NJN,Away,Win,0.0,95.0,...,94.6,105.7,-11.1,0.36,17.86,1.20,0.27,89.9,89.9,NJN


In [8]:
team_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15348 entries, 0 to 1142
Columns: 124 entries, gmDate to matchWinner
dtypes: float64(110), int64(4), object(10)
memory usage: 14.6+ MB


# 1.2 Cleaning player_stats

In [9]:
#Reorder columns and naming
col = ['Player', 'Team', 'Against', 'Date']
other = [x for x in player_stats.columns if x not in col]
player_stats= player_stats[col + other]

In [10]:
#Dropping unecessary columns

player_stats.drop(columns = ['FG%', '3P%', 'FT%', 'GameLink'], inplace = True)

#'GameLink' turns out to be strings with no interesting information. If the full link was available, possible use to create Game IDs based on URL, however, that is not the case, and this column should be removed as it provides nothing of value.

In [11]:
#Checking for isnull()

player_stats.isna().sum()

Player         0
Team           0
Against        0
Date           0
Home           0
MP         61743
FG         61743
FGA        61743
3P         61743
3PA        61743
FT         61743
FTA        61743
ORB        61743
DRB        61743
TRB        61743
AST        61743
STL        61743
BLK        61743
TOV        61743
PF         61743
PTS        61743
+/-        61793
dtype: int64

While evaluating the NA values, we see that we do have a good chunk of NaN data. It is clear that we are not looking at players who received 0 stats sucha as 0 pts or rebounds. Instead the data clearly records players who played zero mins have NaN's in their stats. 

In the +/- column, there's always a chance that the player contributed, but their +/- is still 0. This is quite possible as +/- is a stat recorded based on both team performances during the time the player is playing. 

In [12]:
player_stats = player_stats[player_stats['MP'].notna()]

In [13]:
player_stats.isna().sum()

Player      0
Team        0
Against     0
Date        0
Home        0
MP          0
FG          0
FGA         0
3P          0
3PA         0
FT          0
FTA         0
ORB         0
DRB         0
TRB         0
AST         0
STL         0
BLK         0
TOV         0
PF          0
PTS         0
+/-        50
dtype: int64

In [14]:
#Converting 'Date' column to datetime object

player_stats['Date'] = pd.to_datetime(player_stats['Date'])

In [15]:
player_stats[player_stats['Home'] == 1]

,Player,Team,Against,Date,Home,MP,FG,FGA,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
12,Ray Allen,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,40.39,2.0,9.0,1.0,4.0,...,1.0,3.0,4.0,1.0,1.0,0.0,4.0,1.0,8.0,-1.0
13,Paul Pierce,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,38.13,10.0,19.0,2.0,4.0,...,1.0,2.0,3.0,4.0,1.0,0.0,2.0,4.0,27.0,12.0
14,Kevin Garnett,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,36.09,5.0,15.0,0.0,0.0,...,1.0,5.0,6.0,2.0,2.0,0.0,2.0,1.0,11.0,5.0
15,Rajon Rondo,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,29.57,4.0,5.0,0.0,0.0,...,1.0,4.0,5.0,6.0,3.0,0.0,2.0,5.0,14.0,4.0
16,Kendrick Perkins,Boston Celtics,Cleveland Cavaliers,2008-10-28,1,20.50,1.0,2.0,0.0,0.0,...,4.0,4.0,8.0,0.0,0.0,0.0,2.0,6.0,2.0,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27735,Kelly Olynyk,Miami Heat,Los Angeles Lakers,2020-10-11,1,14.52,4.0,7.0,1.0,1.0,...,3.0,4.0,7.0,0.0,0.0,1.0,0.0,1.0,9.0,19.0
27736,Kendrick Nunn,Miami Heat,Los Angeles Lakers,2020-10-11,1,12.45,3.0,8.0,2.0,4.0,...,0.0,3.0,3.0,1.0,0.0,0.0,2.0,0.0,8.0,-13.0
27737,Andre Iguodala,Miami Heat,Los Angeles Lakers,2020-10-11,1,10.33,0.0,2.0,0.0,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-20.0
27738,Solomon Hill,Miami Heat,Los Angeles Lakers,2020-10-11,1,2.29,2.0,2.0,1.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,5.0,7.0


In [16]:
#Creating game ID column for bridging the team_stats and player_stats datasets.
#We want to uniform the ID to just using the home team abbreviation

if (player_stats['Home'] == 1).all():
    player_stats['homeAbbr'] = player_stats['Team'].str[0:3].str.upper()
else:
    player_stats['homeAbbr'] = player_stats['Against'].str[0:3].str.upper()

player_stats['homeAbbr'].unique()

#We can see that using just the first 3 characters most of the abbreviations are the standard for representing the NBA teams, but there are exceptions. 
#Need to change (LOS, NEW, SAN, OKL, GOL, BRO)

array(['BOS', 'CLE', 'CHI', 'MIL', 'LOS', 'POR', 'PHI', 'TOR', 'WAS',
       'NEW', 'ORL', 'ATL', 'MIA', 'SAN', 'PHO', 'OKL', 'MIN', 'SAC',
       'DET', 'IND', 'HOU', 'MEM', 'UTA', 'DEN', 'GOL', 'CHA', 'DAL',
       'BRO'], dtype=object)

In [17]:
player_stats['homeAbbr'] = player_stats['homeAbbr'].replace({'LOS': 'LAL', 'NEW': 'NYK', 'SAN':'SAS', 'OKL': 'OKC', 'GOL':'GSW', 'BRO':'BKN'})
player_stats['homeAbbr'].unique()

array(['BOS', 'CLE', 'CHI', 'MIL', 'LAL', 'POR', 'PHI', 'TOR', 'WAS',
       'NYK', 'ORL', 'ATL', 'MIA', 'SAS', 'PHO', 'OKC', 'MIN', 'SAC',
       'DET', 'IND', 'HOU', 'MEM', 'UTA', 'DEN', 'GSW', 'CHA', 'DAL',
       'BKN'], dtype=object)

In [18]:
player_stats['gameID'] =  player_stats['Date'].dt.strftime('%Y%m%d') + player_stats['homeAbbr']
player_stats.head()

,Player,Team,Against,Date,Home,MP,FG,FGA,3P,3PA,...,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,homeAbbr,gameID
0,LeBron James,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,36.00,9.0,21.0,0.0,4.0,...,7.0,6.0,2.0,1.0,3.0,4.0,22.0,-3.0,BOS,20081028BOS
1,Delonte West,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,35.29,2.0,4.0,0.0,1.0,...,2.0,2.0,0.0,0.0,1.0,0.0,6.0,-10.0,BOS,20081028BOS
2,Mo Williams,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,31.18,4.0,10.0,3.0,5.0,...,3.0,2.0,1.0,0.0,4.0,5.0,12.0,3.0,BOS,20081028BOS
3,Zydrunas Ilgauskas,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,30.24,3.0,5.0,0.0,0.0,...,8.0,1.0,0.0,0.0,4.0,3.0,15.0,7.0,BOS,20081028BOS
4,Ben Wallace,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,19.03,1.0,1.0,0.0,0.0,...,4.0,0.0,0.0,1.0,1.0,2.0,2.0,-1.0,BOS,20081028BOS


In [19]:
#Checking if games included playoff games. In the 2008 season, the playoffs started April 20th
#df1[df1['col'].str.contains(r'foo(?!$)')]
player_stats[player_stats['gameID'].str.contains(r'200905(?!$)')]

#Below data looks like the player stats include playoff games. 
#We are only concerned with regular season, and will need to remove these games based on dates

start_date = '2009-04-18'
end_date = '2009-06-19'
after = player_stats['Date'] >= start_date
before = player_stats['Date'] <= end_date
playoff0809 = after & before

start_date2 = '2010-04-17'
end_date2 = '2010-06-17'
after2 = player_stats['Date'] >= start_date
before2 = player_stats['Date'] <= end_date
playoff0910 = after2 & before2

start_date3 = '2011-04-16'
end_date3 = '2011-06-12'
after3 = player_stats['Date'] >= start_date
before3 = player_stats['Date'] <= end_date
playoff1011 = after3 & before3

start_date4 = '2012-04-28'
end_date4 = '2012-06-21'
after4 = player_stats['Date'] >= start_date
before4 = player_stats['Date'] <= end_date
playoff1112 = after4 & before4

start_date5 = '2013-04-20'
end_date5 = '2013-06-20'
after5 = player_stats['Date'] >= start_date
before5 = player_stats['Date'] <= end_date
playoff1213 = after5 & before5

start_date6 = '2014-04-19'
end_date6 = '2014-06-14'
after6 = player_stats['Date'] >= start_date
before6 = player_stats['Date'] <= end_date
playoff1314 = after6 & before6

start_date7 = '2015-04-18'
end_date7 = '2015-06-16'
after7 = player_stats['Date'] >= start_date
before7 = player_stats['Date'] <= end_date
playoff1415 = after7 & before7

start_date8 = '2016-04-16'
end_date8 = '2016-06-19'
after8 = player_stats['Date'] >= start_date
before8 = player_stats['Date'] <= end_date
playoff1516 = after8 & before8

start_date9 = '2017-04-15'
end_date9 = '2017-06-12'
after9 = player_stats['Date'] >= start_date
before9 = player_stats['Date'] <= end_date
playoff1617 = after9 & before9

start_date10 = '2018-04-14'
end_date10 = '2018-06-8'
after10 = player_stats['Date'] >= start_date
before10 = player_stats['Date'] <= end_date
playoff1718 = after10 & before10

start_date11 = '2019-04-13'
end_date11 = '2019-06-13'
after11 = player_stats['Date'] >= start_date
before11 = player_stats['Date'] <= end_date
playoff1819 = after11 & before11

start_date12 = '2020-08-17'
end_date12 = '2020-10-11'
after12 = player_stats['Date'] >= start_date
before12 = player_stats['Date'] <= end_date
playoff1920 = after12 & before12

playoffdates = [playoff0809, playoff0910, playoff1011, playoff1112, playoff1213, playoff1314, playoff1415, playoff1516, playoff1617, playoff1718, playoff1819, playoff1920]
playoffgames = pd.concat(playoffdates)


In [ ]:
player_stats[playoffgames].head()

<ipython-input-38-8066c5abf08a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  player_stats[playoffgames].head()


In [20]:
player_stats.drop(columns = ['homeAbbr'])

,Player,Team,Against,Date,Home,MP,FG,FGA,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,gameID
0,LeBron James,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,36.00,9.0,21.0,0.0,4.0,...,6.0,7.0,6.0,2.0,1.0,3.0,4.0,22.0,-3.0,20081028BOS
1,Delonte West,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,35.29,2.0,4.0,0.0,1.0,...,2.0,2.0,2.0,0.0,0.0,1.0,0.0,6.0,-10.0,20081028BOS
2,Mo Williams,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,31.18,4.0,10.0,3.0,5.0,...,3.0,3.0,2.0,1.0,0.0,4.0,5.0,12.0,3.0,20081028BOS
3,Zydrunas Ilgauskas,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,30.24,3.0,5.0,0.0,0.0,...,5.0,8.0,1.0,0.0,0.0,4.0,3.0,15.0,7.0,20081028BOS
4,Ben Wallace,Cleveland Cavaliers,Boston Celtics,2008-10-28,0,19.03,1.0,1.0,0.0,0.0,...,4.0,4.0,0.0,0.0,1.0,1.0,2.0,2.0,-1.0,20081028BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27735,Kelly Olynyk,Miami Heat,Los Angeles Lakers,2020-10-11,1,14.52,4.0,7.0,1.0,1.0,...,4.0,7.0,0.0,0.0,1.0,0.0,1.0,9.0,19.0,20201011LAL
27736,Kendrick Nunn,Miami Heat,Los Angeles Lakers,2020-10-11,1,12.45,3.0,8.0,2.0,4.0,...,3.0,3.0,1.0,0.0,0.0,2.0,0.0,8.0,-13.0,20201011LAL
27737,Andre Iguodala,Miami Heat,Los Angeles Lakers,2020-10-11,1,10.33,0.0,2.0,0.0,2.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-20.0,20201011LAL
27738,Solomon Hill,Miami Heat,Los Angeles Lakers,2020-10-11,1,2.29,2.0,2.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,5.0,7.0,20201011LAL


In [21]:
clean_player_stats = player_stats

In [22]:
clean_player_stats.shape

(311326, 24)

# 1.3 Cleaning team_stats

In [23]:
team_stats.columns.tolist()

['gmDate',
 'seasonType',
 'season',
 'teamWins',
 'teamLosses',
 'teamAbbr',
 'teamLoc',
 'teamRslt',
 'teamDayOff',
 'teamPTS',
 'teamAST',
 'teamTO',
 'teamMin',
 'teamSTL',
 'teamBLK',
 'teamPF',
 'teamFGA',
 'teamFGM',
 'teamFG%',
 'team2PA',
 'team2PM',
 'team2P%',
 'team3PA',
 'team3PM',
 'team3P%',
 'teamFTA',
 'teamFTM',
 'teamFT%',
 'teamORB',
 'teamDRB',
 'teamTRB',
 'teamPTS1',
 'teamPTS2',
 'teamPTS3',
 'teamPTS4',
 'teamPTS5',
 'teamPTS6',
 'teamPTS7',
 'teamPTS8',
 'teamPTS9',
 'teamPTS10',
 'teamTREB%',
 'teamASST%',
 'teamTS%',
 'teamEFG%',
 'teamOREB%',
 'teamDREB%',
 'teamTO%',
 'teamSTL%',
 'teamBLK%',
 'teamBLKR',
 'teamPPS',
 'teamFIC',
 'teamFIC40',
 'teamOrtg',
 'teamDrtg',
 'teamEDiff',
 'teamPlay%',
 'teamAR',
 'teamPoss',
 'teamAST/TO',
 'teamPace',
 'teamSTL/TO',
 'opptWins',
 'opptLosses',
 'opptAbbr',
 'opptLoc',
 'opptRslt',
 'opptDayOff',
 'opptPTS',
 'opptAST',
 'opptTO',
 'opptMin',
 'opptSTL',
 'opptBLK',
 'opptPF',
 'opptFGA',
 'opptFGM',
 'opptFG%',

In [24]:
#Our analysis will focus on regular season for the recreating portion of our anaylysis. 
team_stats['seasonType'].unique()

#We will separate the data into regular season and playoffs.

team_stats = team_stats[team_stats['seasonType']=='Regular']

In [25]:
team_stats['seasonType'].unique()

array(['Regular'], dtype=object)

In [26]:
#The games stats are recorded as team vs. opponent with another column indicating whether this is a home game or an away game. 
#This format leaves me to suspect there is double data of the same game. Let's investigate. 

team_stats['gmDate'] = pd.to_datetime(team_stats['gmDate'])
team_stats[['teamAbbr', 'teamLoc', 'opptAbbr', 'opptLoc']][team_stats['gmDate'] == datetime(2008,10,29)]

#Seems our suspicion was wrong and away_games=team as home_games=opp. We will keep this in mind during feature engineering. 

,teamAbbr,teamLoc,opptAbbr,opptLoc
3,TOR,Away,PHI,Home
4,NJN,Away,WAS,Home
5,ATL,Away,ORL,Home
6,MIA,Away,NYK,Home
7,PHO,Away,SAS,Home
8,MIL,Away,OKC,Home
9,SAC,Away,MIN,Home
10,IND,Away,DET,Home
11,MEM,Away,HOU,Home
12,DEN,Away,UTA,Home


In [27]:
team_stats['gmDate'] = pd.to_datetime(team_stats['gmDate'])
team_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14349 entries, 0 to 1064
Columns: 124 entries, gmDate to matchWinner
dtypes: datetime64[ns](1), float64(110), int64(4), object(9)
memory usage: 13.7+ MB


In [28]:
dateString =  team_stats['gmDate'].dt.strftime('%Y%m%d')
dateString.head()

0    20081028
1    20081028
2    20081028
3    20081029
4    20081029
Name: gmDate, dtype: object

In [29]:
team_stats['GameID'] = dateString + team_stats['opptAbbr']
team_stats['GameID'].head()

0    20081028BOS
1    20081028CHI
2    20081028LAL
3    20081029PHI
4    20081029WAS
Name: GameID, dtype: object

In [30]:
team_stats[['GameID','teamAbbr', 'teamLoc', 'opptAbbr', 'opptLoc']][team_stats['gmDate'] == datetime(2015,10,29)]

,GameID,teamAbbr,teamLoc,opptAbbr,opptLoc
17,20151029IND,MEM,Away,IND,Home
18,20151029NYK,ATL,Away,NYK,Home
19,20151029LAC,DAL,Away,LAC,Home


In [31]:
#Renaming Columns

team_stats.rename(columns = {'gmDate' : 'Date', 'teamAbbr' : 'team', 'opptAbbr' : 'opponent' ,'teamFGM': 'teamFGMade', 'team2PM' : 'team2PMade' , 'team3PM' : 'team3PMade', 'teamFTM' : 'teamFTMade',   'opptFGM': 'opptFGMade', 'oppt2PM': 'oppt2PMade', 'oppt3PM' : 'oppt3PMade', 'opptFTM' : 'opptFTMade'}, inplace = True)

In [32]:
#Removing percentage columns and 

team_stats.drop(columns = ['seasonType', 'teamLoc', 'team2P%', 'team3P%', 'teamFT%', 'teamTREB%', 'teamASST%', 'teamTS%', 'teamEFG%', 'teamOREB%', 'teamDREB%', 'teamTO%', 'teamSTL%', 'teamBLK%', 'teamBLKR', 'teamPPS', 'teamFIC', 'teamFIC40' , 'teamPlay%', 'teamAR', 'teamPoss', 'teamPace', 'teamSTL/TO', 'opptLoc', 'oppt2P%', 'oppt3P%', 'opptFT%', 'opptTREB%', 'opptASST%', 'opptTS%', 'opptEFG%', 'opptOREB%', 'opptDREB%', 'opptTO%', 'opptSTL%', 'opptBLK%', 'opptBLKR', 'opptPPS', 'opptFIC', 'opptFIC40', 'opptPlay%', 'opptAR', 'opptSTL/TO', 'opptPoss', 'opptPace'], inplace = True)

In [33]:
#Creating Missed Columns

team_stats['teamFGMissed'] = team_stats['teamFGA'] - team_stats['teamFGMade']
team_stats['team2PMissed'] = team_stats['team2PA'] - team_stats['team2PMade'] 
team_stats['team3PMissed'] = team_stats['team3PA'] - team_stats['team3PMade']
team_stats['opptFGMissed'] = team_stats['opptFGA'] - team_stats['opptFGMade']
team_stats['oppt2PMissed'] = team_stats['oppt2PA'] - team_stats['oppt2PMade']
team_stats['oppt3PMissed'] = team_stats['oppt3PA'] - team_stats['oppt3PMade']

In [34]:
team_stats.head()

,Date,season,teamWins,teamLosses,team,teamRslt,teamDayOff,teamPTS,teamAST,teamTO,...,opptEDiff,opptAST/TO,matchWinner,GameID,teamFGMissed,team2PMissed,team3PMissed,opptFGMissed,oppt2PMissed,oppt3PMissed
0,2008-10-28,2008-09,0,1,CLE,Loss,0.0,85.0,16.0,21.0,...,5.4,1.00,BOS,20081028BOS,39.0,27.0,12.0,41.0,30.0,11.0
1,2008-10-28,2008-09,0,1,MIL,Loss,0.0,95.0,21.0,19.0,...,14.5,1.50,CHI,20081028CHI,41.0,33.0,8.0,34.0,27.0,7.0
2,2008-10-28,2008-09,0,1,POR,Loss,0.0,76.0,19.0,12.0,...,22.4,1.27,LAL,20081028LAL,55.0,41.0,14.0,42.0,39.0,3.0
3,2008-10-29,2008-09,1,0,TOR,Win,0.0,95.0,24.0,9.0,...,-13.0,0.88,TOR,20081029PHI,45.0,39.0,6.0,55.0,40.0,15.0
4,2008-10-29,2008-09,1,0,NJN,Win,0.0,95.0,22.0,9.0,...,-11.1,1.20,NJN,20081029WAS,41.0,28.0,13.0,51.0,35.0,16.0


In [35]:
clean_team_stats = team_stats

# Section 2 Feature Engineering

Our main goal is to predict winning and losing of games between teams. The stats teams and players accrue in games are not as helpful, therefore, we will engineer a few features that will provide more insight into win/loss predictions. The prior investigation utilized the following. 

1. Elo Ratings 
The Elo system is a calculate stat that keeps track of relative skill and quality of teams within the league. Everyteam starts with the same Elo rating at the beggining of a season, and will either add to or subtract value as they win/lose throughoput the season. 

2. Recent Team Performances 
We will keep track of the rolling averages for a teams performance over the last 10 games prior to the game we are predicting. This gives us a glimps of how the team is playing recently leading up to the predicted game. 

3. Recent Player Performances 
Keeping track of player performances can also tell us how well that player will play (suspicious of paisson model behaviour for players and their scoring in view of their seasonal stats) 

4. Player Performance In Season
Likewise, how this player has performed up to their current game within the season can also gives us glimpse into how their performances are trending. 

# 2.2 Elo Ratings

Elo rating takes into account variables suc as difficult of schedule, home court advantage, margin of victory, and etc. These variables give more insight than simply utilizing team win record percentages. The formula below was concepted by [Nate Silver](https://fivethirtyeight.com/features/how-we-calculate-nba-elo-ratings/#:~:text=Here's%20the%20formula%3A%20Take%20the,and%20then%20divide%20by%2028.). 

All teams start off with a 1500 elo rating. After each game, the winner is award addition elo value and the loser has their elo value subtracted. The equation to calculate for Elo is shown below, where $R_{i}$ is the current Elo rating.

$$ R_{i + 1} = k * (S_{team} - E_{team} + R_{i}) $$


$S_{team} = 1 $ when the team wins, and 0 if the team loses. The $E_{team}$ value is expected win probability of the team which is further defined as 

$$ E_{team} = \frac{1}{1+10^{\frac{opp\_elo - team\_elo}{400}}}$$

The $k$ in the the Elo equationis a moving constant depending on the margin of victory and difference in Elo ratings. The k constant is calculated as: 

$$ k = 20 \frac{(MOV_{winner} + 3)^{0.8}}{7.5 + 0.006(elo\_difference_{winner})} $$

Elo ratings carry over from season to season, as teams who were good previous season, usually do not have significant drop offs from previous seasons. If $R$ is a teams final Elo rating for a season, than below is the Elo rating to being their next season. 

$$ (R * 0.75) + (0.25 * 0501)$$

## Def Elo Variable Functions

Function for win probabilty as per Nate Silver [Win Prob](https://fivethirtyeight.com/features/introducing-nfl-elo-ratings/)

$$ Pr(Home) = \frac{1}{10^{(-EloDiff/400)}+1}$$

In [36]:
#Define function to calculate Win Probability implied by Elo ratings and home court adjustments
def win_probs(home_elo, away_elo, home_court_advantage) :
    h = math.pow(10, home_elo/400)
    r = math.pow(10, away_elo/400)
    a = math.pow(10, home_court_advantage/400)
    
    denom = r + a*h
    home_prob = a*h / denom
    away_prob = r / denom
    
    return home_prob, away_prob

#odds the home team will win based on elo ratings and home court advantage

def home_odds_on(home_elo, away_elo, home_court_advantage) :
    h = math.pow(10, home_elo/400)
    r = math.pow(10, away_elo/400)
    a = math.pow(10, home_court_advantage/400)
    
    return a*h/r

#this function determines the constant used in the elo rating, based on margin of victory and difference in elo ratings
def elo_k(MOV, elo_diff):
    k = 20
    if MOV>0:
        multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
    else:
        multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
    return k*multiplier

#updates the home and away teams elo ratings after a game 

def update_elo(home_score, away_score, home_elo, away_elo, home_court_advantage) :
    home_prob, away_prob = win_probs(home_elo, away_elo, home_court_advantage) 
    
    if (home_score - away_score > 0) :
        
        home_win = 1 
        away_win = 0 
    
    else :
        
        home_win = 0 
        away_win = 1 

k = elo_k(home_score - away_score, home_elo - away_elo)

updated_home_elo = home_elo + k * (home_win - home_prob) 
updated_away_elo = away_elo + k * (away_win - away_prob)

return updated_home_elo, updated_away_elo


#takes into account prev season elo
def get_prev_elo(team, date, season, team_stats, elo_df) :
    prev_game = team_stats[team_stats['Date'] < game_date][(team_stats['H_Team'] == team) | (team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(1).iloc[0] 
    
    if team == prev_game['H_Team'] :
        elo_rating = elo_df[elo_df['Game_ID'] == prev_game['Game_ID']]['H_Team_Elo_After'].values[0]
        
    else :
        elo_rating = elo_df[elo_df['Game_ID'] == prev_game['Game_ID']]['A_Team_Elo_After'].values[0]

if prev_game['Season'] != season :
    return (0.75 * elo_rating) + (0.25 * 1505)
else :
    return elo_rating


NameError: name 'home_score' is not defined

## Calculating Elo Ratings

In [ ]:
team_stats.sort_values(by = 'Date', inplace = True)
team_stats.reset_index(inplace=True, drop = True)

elo_df = pd.DataFrame(columns=['Game_ID', 'H_Team', 'A_Team', 'H_Team_Elo_Before', 'A_Team_Elo_Before', 'H_Team_Elo_After', 'A_Team_Elo_After'])

# Previously version work to be sorted...

In [ ]:
#website - "https://www.kaggle.com/heeebsinc/nbaseasonstats201820"
#The above datasets are split into season from 2000 to 2020. We are only interested in 2008 - 2020.
cs_player= pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/player_stats/2008-2009.csv')

In [ ]:
cs_player.info()

In [ ]:
cs_player.head()

In [ ]:
print(cs_player['GameLink'])

In [ ]:
#Code for removing columns for future reference. 
#cs_player = cs_player.drop(['GameLink'], axis=1)

In [ ]:
#Question, when trying to add seasons to the dataset, it seems to immediately jump to 2019 instead of following
#the start year as noted below.
start_year = 2008
end_year = 2009

player_stats_combined_df = pd.DataFrame()

for i in range(12) :
  player_stats_file_path = '/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/player_stats/' + str(start_year) + "-" + str(end_year) + '.csv'

  player_stats_combined_df = player_stats_combined_df.append(pd.read_csv(player_stats_file_path))

  start_year = start_year + 1
  end_year = end_year + 1

In [ ]:
player_stats_combined_df.head()

In [ ]:
#set(player_stats_combined_df['Season'])

In [ ]:
player_stats_combined_df.info()

In [ ]:
player_stats_combined_df.describe()

In [ ]:
player_stats_combined_df.isnull().sum()

In [ ]:
player_stats_combined_df[player_stats_combined_df['3P%'].isnull()]

In [ ]:
threeptking = player_stats_combined_df[['Player','Team', 'Against', 'Date', '3PA', 'PTS']][player_stats_combined_df['3P%']==1]
threeptking[threeptking['3PA']>8]

In [ ]:
print(player_stats_combined_df[['Player','Date','3PA','Against','3P%']][player_stats_combined_df['3PA']>8])

In [ ]:
player_stats_combined_df['Ast/To'] = player_stats_combined_df['AST']/player_stats_combined_df['TOV']

In [ ]:
player_stats_combined_df = player_stats_combined_df.drop(['GameLink'], axis = 1)

In [ ]:
player_stats_combined_df['Date'] = pd.to_datetime(player_stats_combined_df['Date'])

In [ ]:
player_stats_combined_df['Date']

In [ ]:
#Dataset acquired from kaggle: 
#website - "https://www.kaggle.com/rafaelgreca/nba-games-box-score-since-1949"

cs_team = pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/team_stats/2019-2020.csv')

In [ ]:
cs_team.info()

In [ ]:
print(cs_team.columns.tolist())

In [ ]:
cs_team.head()

In [ ]:
start_year = 2008
end_year = 2009

team_stats_combined_df = pd.DataFrame()

for i in range(12) :
  team_stats_file_path = '/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/team_stats/' + str(start_year) + "-" + str(end_year) + '.csv'
  
  team_stats_combined_df = team_stats_combined_df.append(pd.read_csv(team_stats_file_path, index_col = 0))
  
  start_year = start_year + 1
  end_year = end_year + 1

In [ ]:
team_stats_combined_df.describe()

In [ ]:
team_stats_combined_df.isnull().sum()!=0

In [ ]:
team_stats_combined_df.columns[team_stats_combined_df.isnull().any()].tolist()

In [ ]:
#Dataset from prior investigation - NBA Combined Player Stats
#website - "https://towardsdatascience.com/predicting-the-outcome-of-nba-games-with-machine-learning-a810bb768f20"

ps_player = pd.read_csv('NBA_Combined_Player_Stats.csv', index_col= 0)

In [ ]:
ps_player.head()

In [ ]:
ps_player.info()

In [ ]:
## 2.2 REVIEWING PRIOR INVESTIGATION'S TEAM STATISTICS 

In [ ]:
#Dataset from prior investigation - NBA Combined Team Stats 
#website - "https://towardsdatascience.com/predicting-the-outcome-of-nba-games-with-machine-learning-a810bb768f20"

ps_team = pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/NBA_Combined_Team_Stats.csv', index_col=0)

In [ ]:
ps_team.info()

In [ ]:
ps_team.head()

In [ ]:
ps_team.columns.to_list()